In [2]:
import pandas as pd
#from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense,SpatialDropout1D
from tensorflow.keras.layers import LSTM, Embedding
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import spacy
from sklearn.preprocessing import MaxAbsScaler
from sklearn.multiclass import OneVsRestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline, FeatureUnion, make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_curve
from sklearn.feature_selection import chi2, SelectKBest

In [3]:
#df=pd.read_csv('https://raw.githubusercontent.com/kushalchauhan98/bcn-cnn-text-classification/master/hm_train.csv')
#df.to_csv('hm_train.csv')
df=pd.read_csv('hm_train.csv')

In [4]:
filter=df['cleaned_hm'].str.len()<=100
df=df[filter]

In [5]:
print(df["cleaned_hm"])

0        I went on a successful date with someone I fel...
1        I was happy when my son got 90% marks in his e...
2             I went to the gym this morning and did yoga.
4        I went with grandchildren to butterfly display...
5                                  I meditated last night.
                               ...                        
60315              when i got a high level on a video game
60316    I got together with my best friend and baked c...
60317                  I went to a restaurant with friends
60318    The other day on Mechanical Turk I made over f...
60319    Finished the semester today and aced majority ...
Name: cleaned_hm, Length: 43354, dtype: object


In [6]:
#rint(df)
y=df['predicted_category']

y = pd.get_dummies(df['predicted_category']).values
print('Shape of label tensor:', y.shape)

LABELS=sorted(df['predicted_category'].unique())

print(y)
print(LABELS)

# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 50000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 250
# This is fixed.
EMBEDDING_DIM = 100

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(df['cleaned_hm'].values)
word_index = tokenizer.word_index

X = tokenizer.texts_to_sequences(df['cleaned_hm'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.10, random_state = 42)



Shape of label tensor: (43354, 7)
[[0 1 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 0 0 ... 1 0 0]
 ...
 [0 0 1 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]]
['achievement', 'affection', 'bonding', 'enjoy_the_moment', 'exercise', 'leisure', 'nature']
Shape of data tensor: (43354, 250)


In [7]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(len(LABELS), activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
epochs = 5
batch_size = 64

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])


Epoch 1/5
106/549 [====>.........................] - ETA: 5:13 - loss: 1.4370 - accuracy: 0.4800

In [ ]:
accr = model.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

In [ ]:
plt.title('Loss')
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show();

In [ ]:
def GetCategory(pred,LABELS):
    returnVal=[]
    curr_index=0
    curr_val=0
    for item in pred:
        val=max(item)
        index=list(item).index(val)
        #print(val,LABELS[index])
        if val>curr_val:
            curr_index=index
            curr_val=val
            
    return LABELS[curr_index]
    #returnVal.append(LABELS[np.argmax(np.array(item))])
    #return returnVal
        
for key,item in df.iterrows():
    data=[]
    data.append(item['cleaned_hm'])
    seq = tokenizer.texts_to_sequences(data)
    padded = pad_sequences(seq, maxlen=MAX_SEQUENCE_LENGTH)
    pred = model.predict(padded)
    print(pred)
    category=LABELS[np.argmax(pred)]
    print(data, "[predicted category]", category, "[actual category]",item['predicted_category'])
    #break
    #print(pred, LABELS[np.argmax(pred)])

In [ ]:
TEXT=['cleaned_hm']
TARGET='predicted_category'
NUMERIC=[]
#def combine_text_columns(data_frame, to_drop=NUMERIC + LABELS):
def combine_text_columns(data_frame, text_labels=TEXT):
    """ converts all text in each row of data_frame to single vector """
    # Drop non-text columns that are in the df
    #to_drop = set(to_drop) & set(data_frame.columns.tolist())
    text_data =data_frame[text_labels]
     
    # Replace nans with blanks
    text_data.fillna("",inplace=True)
    
    # Join all text items in a row that have a space in between
    return text_data.apply(lambda x: " ".join(x), axis=1)

In [ ]:
nlp = spacy.load('en_core_web_sm')
stopwords=spacy.lang.en.stop_words.STOP_WORDS
chi_k = 300
# Create the token pattern: TOKENS_ALPHANUMERIC
TOKENS_ALPHANUMERIC = '[A-Za-z0-9]+(?=\\s+)'

get_text_data = FunctionTransformer(combine_text_columns,validate=False)

# Preprocess the numeric data: get_numeric_data
#get_numeric_data = FunctionTransformer(lambda x: x[NUMERIC], validate=False)

pl = Pipeline([
        ('union', FeatureUnion(
            transformer_list = [
                #('numeric_features', Pipeline([
                #    ('selector', get_numeric_data),
                #    ('imputer', SimpleImputer())
                #])),
                ('text_features', Pipeline([
                    ('selector', get_text_data),
                    ('vectorizer', TfidfVectorizer(stop_words='english')),
                    #('vectorizer',CountVectorizer(stop_words=stopwords,token_pattern=TOKENS_ALPHANUMERIC, ngram_range=(1,2))),('dim_red', SelectKBest(chi2, chi_k))
                ]))
             ]
        )),
        ('scale', MaxAbsScaler()),
        ('clf', OneVsRestClassifier(RandomForestClassifier(n_estimators=15))),
        #('nb', MultinomialNB()),
        #('lr', OneVsRestClassifier(LogisticRegression(C=100)))
    ])

In [ ]:
X=df[TEXT]
y=df[TARGET]

#print(X)

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=.3, random_state=42)

pl.fit(X_train,y_train)
predictions=pl.predict(X_train)

accuracy = pl.score(X_train, y_train)
print("\nAccuracy:{:.2f}% ".format(accuracy))



In [ ]:
predictions=pl.predict(X_test)

index=0
for key,item in X_test.iterrows():
    #print(item['cleaned_hm'])
    sentence=item['cleaned_hm']
    result=predictions[index]
    #actual=y_test[index]
    index+=1
    print(sentence,"[[result]]",result)

In [ ]:
print("done")